In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from scipy.stats import shapiro
from pingouin import multivariate_normality

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
df = pd.read_excel('Exercise.xlsx')
df.columns = list(df.iloc[0, :].values)
df.drop(index=0, inplace=True)
df.drop('Name', axis=1, inplace=True)
df.head()

,Gender,Age,Height(cm),Weight(kg),Lifestyle,Exercise,Daily Calories Intake,Target,How motivated are you for being healthy\n(in %),Body Mass Index (BMI),Basal Metabolic Rate (BMR),Type,Body Fat Percentage (BFP),Body Type,Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie),Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie),Calorie Difference (Daily Calorie Intake - TDEE),Suggestion
1,Male,18,152,50,1. Sedentary or light activity (e.g. - Office ...,1. Sedentary (little or no exercise),2067.152431,Loose Weight,80,21.641274,1392.866,Boy,17.878324,Normal,1671.4392,2131.08498,-0.03,LOW CARB DIET (25:35:40)
2,Male,18,152,54,1. Sedentary or light activity (e.g. - Office ...,2. Lightly active (light exercise/sports 1-3 d...,1705.730935,Maintain Weight,78,23.372576,1447.866,Boy,20.49259,Normal,1990.81575,2215.23498,-0.23,BALANCE DIET (55:25:20)
3,Male,18,152,58,1. Sedentary or light activity (e.g. - Office ...,3. Moderately active (moderate exercise/sports...,1910.142686,Gain Muscle,65,25.103878,1502.866,Boy,23.106856,Overweight,2329.4423,2299.38498,-0.18,BALANCE DIET (55:25:20)
4,Male,18,152,62,3. Vigorously active (Agricultural worker (non...,4. Very active (hard exercise/sports 6-7 days ...,2552.952908,Loose Weight,82,26.83518,1557.866,Boy,25.721122,Overweight,2687.31885,3505.1985,-0.271667,KETOGENIC DIET (5:35:60)
5,Male,18,152,66,1. Sedentary or light activity (e.g. - Office ...,1. Sedentary (little or no exercise),1826.086885,Maintain Weight,75,28.566482,1612.866,Boy,28.335388,Overweight,1935.4392,2467.68498,-0.26,BALANCE DIET (55:25:20)


In [25]:
# Mapping String to numeric values.
df['Gender']=df['Gender'].map({"Male":0,"Female":1})
df['Body Type']=df['Body Type'].map({"Underweight":0,"Normal":1,"Overweight":2,"Obese":3})
df['Type']=df['Type'].map({"Boy":0,"Girl":1,"Adult Male":2,"Adult Female":3})
df['Suggestion']=df['Suggestion'].map({"LOW CARB DIET (25:35:40)":0,"BALANCE DIET (55:25:20)":1,"ZONE DIET (40:30:30)":2,"KETOGENIC DIET (5:35:60)":3,"DEPLETION DIET (DYNAMIC)":4,"HIGH CARB DIET(60:20:20)":5})
df['Target']=df['Target'].map({"Loose Weight":0,"Maintain Weight":1,"Gain Muscle":2})
df['Lifestyle']=df['Lifestyle'].map({"1. Sedentary or light activity (e.g. - Office worker getting little or no exercise)":0,"2. Active or moderately active (Construction worker or person running one hour daily)":1,"3. Vigorously active (Agricultural worker (non mechanized) or person swimming two hours daily)":2})
df['Exercise']=df['Exercise'].map({"1. Sedentary (little or no exercise)":0,"2. Lightly active (light exercise/sports 1-3 days/ week)":1,"3. Moderately active (moderate exercise/sports 3-5days/week)":2,"4. Very active (hard exercise/sports 6-7 days a week)":3})

In [26]:
df = df.apply(pd.to_numeric)

In [27]:
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

print(to_drop)

# Drop features 
df.drop(to_drop, axis=1, inplace=True)

['Body Type']


In [28]:
## checking correlation after normalizing the data
df_scaled = StandardScaler().fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled

,Gender,Age,Height(cm),Weight(kg),Lifestyle,Exercise,Daily Calories Intake,Target,How motivated are you for being healthy\n(in %),Body Mass Index (BMI),Basal Metabolic Rate (BMR),Type,Body Fat Percentage (BFP),Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie),Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie),Calorie Difference (Daily Calorie Intake - TDEE),Suggestion
0,-1.0,-1.704309,-1.46385,-1.243933,-1.224988,-1.360019,-0.948266,-1.224745,0.237504,-0.592030,-0.541999,-3.618136,-1.298428,-1.199687,-1.055461,-0.168609,-1.113972
1,-1.0,-1.704309,-1.46385,-1.006993,-1.224988,-0.473050,-1.385131,0.000000,0.043670,-0.342490,-0.321641,-3.618136,-1.017369,-0.530306,-0.926927,-1.307818,-0.439284
2,-1.0,-1.704309,-1.46385,-0.770054,-1.224988,0.413919,-1.138050,1.224745,-1.216250,-0.092950,-0.101283,-3.618136,-0.736309,0.179420,-0.798394,-1.023016,-0.439284
3,-1.0,-1.704309,-1.46385,-0.533114,1.224502,1.300887,-0.361060,-1.224745,0.431338,0.156589,0.119074,-3.618136,-0.455250,0.929492,1.043408,-1.545153,0.910093
4,-1.0,-1.704309,-1.46385,-0.296174,-1.224988,-1.360019,-1.239652,0.000000,-0.247081,0.406129,0.339432,-3.618136,-0.174191,-0.646370,-0.541326,-1.478699,-0.439284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,1.0,1.565182,1.46385,0.296174,1.224502,1.300887,1.246541,0.000000,1.109757,-0.404250,-0.455130,0.904534,0.766252,0.411337,0.550863,1.255402,-0.439284
10076,1.0,1.565182,1.46385,0.533114,1.224502,-1.360019,1.136394,1.224745,1.303591,-0.230196,-0.301873,0.904534,0.922045,-1.048948,0.682325,0.913640,2.259470
10077,1.0,1.565182,1.46385,0.770054,1.224502,-0.473050,-0.283890,-1.224745,-0.731665,-0.056142,-0.148616,0.904534,1.077837,-0.405851,0.813787,-1.250857,-1.113972
10078,1.0,1.565182,1.46385,1.006993,1.224502,0.413919,0.337951,0.000000,0.915923,0.117912,0.004641,0.904534,1.233630,0.265307,0.945249,-0.510371,0.235405


## Univariate Analysis
    - Using Shapiro test

In [29]:
def HypoTesting_Uni(pvalue, alpha = 0.05):
    if pvalue < alpha:
        return("Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)")
    else:
        return("Failed to Reject H0 (i.e., The variables follow a univariate normal distribution)*****")

In [34]:
for col in df_scaled.columns:
    test = shapiro(df_scaled[col])
    print("{} - {}".format(col, HypoTesting_Uni(test.pvalue)))

Gender - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Age - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Height(cm) - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Weight(kg) - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Lifestyle - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Exercise - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Daily Calories Intake - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Target - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
How motivated are you for being healthy
(in %) - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Body Mass Index (BMI) - Rejcting H0 (i.e., The variables don't follow a univariate normal distribution)
Basal Metabolic Rate (BMR) - R

## Multivariate Analysis

In [31]:
def HypoTesting_Multi(pvalue, alpha = 0.05):
    if pvalue < alpha:
        return("Rejcting H0 (i.e., The variables don't follow a multivariate normal distribution)")
    else:
        return("Failed to Reject H0 (i.e., The variables follow a multivariate normal distribution)*****")

In [35]:
test_multi = multivariate_normality(df_scaled)
test_multi

HZResults(hz=3.084945844896956, pval=0.0, normal=False)

In [36]:
HypoTesting_Multi(test_multi.pval)

"Rejcting H0 (i.e., The variables don't follow a multivariate normal distribution)"